# USing jieba to cut all answer sentences to words.

In [1]:
import re
import jieba
import pandas as pd

In [2]:
student_answer1_path = '../data/student1ch.xlsx'
reference_answer_path = '../data/reference_answers_extended_ch.xlsx'

In [3]:
def buildWords(str):  
    reg = "[^A-Za-z\u4e00-\u9fa5]"
    words = ' '.join(jieba.cut(str)).split(' ')  
    new = ''
    number = 0
    for word in words:
        word = re.sub(reg, '', word)
        if word is not '':
            number = number + 1
            new = new + word + ' '
        else:
            continue
    if number != 0:
        return new, number
    else:
        return new[:-1], number

In [4]:
student_answer1_df = pd.read_excel(student_answer1_path)
new_colum1 = []
new_colum2 = []
for i in student_answer1_df.index:
    string, number = buildWords(str(student_answer1_df.iloc[i][1]))
    new_colum1.append(string)
    new_colum2.append(number)
student_answer1_df.insert(2,'answer_cut',new_colum1)
student_answer1_df.insert(3,'words_num',new_colum2)
student_answer1_df = student_answer1_df.rename(columns={'格线':'score'})
student_answer1_df.to_csv('../data/student1ch_expand.csv',index=False, encoding='utf-8')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/d4/21c1h0bd5z92rhvl5smhfvw40000gn/T/jieba.cache
Loading model cost 1.219 seconds.
Prefix dict has been built successfully.


In [5]:
reference_answer_df = pd.read_excel(reference_answer_path)
new_colum1 = []
new_colum2 = []
for i in reference_answer_df.index:
    string, number = buildWords(str(reference_answer_df.iloc[i][1]))
    new_colum1.append(string)
    new_colum2.append(number)
reference_answer_df.insert(2,'answer_cut',new_colum1)
reference_answer_df.insert(3,'words_num',new_colum2)
reference_answer_df = reference_answer_df.rename(columns={'格线':'score','refans_text':'refence_text'})
reference_answer_df.to_csv('../data/reference_answers_extended_ch.csv',index=False, encoding='utf-8')

# Make all the setences vectorization and store the results.

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np

Using TensorFlow backend.


In [7]:
import pandas as pd

# Reading the data and store them in DataFrame.
answer_all = pd.read_csv('../data/student1ch_expand.csv', encoding='utf-8')
question_all = pd.read_excel("../data/questions_ch.xlsx")
reference_all = pd.read_csv("../data/reference_answers_extended_ch.csv", encoding='utf-8')

# Extract the students` answers, the question`s describition and references of question 36.
answer_ext = answer_all[answer_all.Question_ID==36]
question_ext = question_all[question_all.Question_ID==36]
reference_ext = reference_all[reference_all.Question_ID==36]

In [8]:
le = LabelEncoder()
data_y = le.fit_transform(answer_ext.score).reshape(-1,1)
ohe = OneHotEncoder()
data_y = ohe.fit_transform(data_y).toarray()

In [9]:
max_words = 2000
max_len = 40
tok = Tokenizer(num_words=max_words)  ## The max word number is 5000
tok.fit_on_texts(answer_ext.answer_cut.astype(str))

In [10]:
data_seq = tok.texts_to_sequences(answer_ext.answer_cut.astype(str))
data_seq_mat = sequence.pad_sequences(data_seq,maxlen=max_len)
data_seq_mat.shape

(249, 40)

In [11]:
reference_seq = tok.texts_to_sequences(reference_ext.answer_cut.astype(str))
reference_seq_mat = sequence.pad_sequences(reference_seq,maxlen=max_len)
reference_seq_mat.shape

(8, 40)

In [12]:
train_seq_mat = data_seq_mat[0:180]
train_y = data_y[0:180]

val_seq_mat = data_seq_mat[180:210]
val_y = data_y[180:210]

test_seq_mat = data_seq_mat[210:]
test_y = data_y[210:]

In [13]:
reference_train_matrix = reference_seq_mat
for i in range(train_seq_mat.shape[0]-1):
    reference_train_matrix = np.row_stack((reference_train_matrix,reference_seq_mat))
pd.DataFrame(reference_train_matrix).to_csv('reference_train.csv', index=False, header=False)

In [14]:
train_matrix = train_seq_mat
for i in range(reference_seq_mat.shape[0]-1):
    train_matrix = np.row_stack((train_matrix,train_seq_mat))
pd.DataFrame(train_matrix).to_csv('train_matrix.csv', index=False, header=False)

In [15]:
train_y_new = train_y
for i in range(reference_seq_mat.shape[0]-1):
    train_y_new = np.row_stack((train_y_new,train_y))
pd.DataFrame(train_y_new).to_csv('train_y_new.csv', index=False, header=False)